# Exercise 07

## Kaggle competition 

* Overview of how Kaggle works ([slides](https://github.com/justmarkham/DAT8/raw/master/slides/16_kaggle.pdf))
* Kaggle Titanic competition: [Titanic: Machine Learning from Disaster](https://www.kaggle.com/c/titanic)

# Exercise 07.1 (20 points)

Create a submission using different classification methods, preprocessing extrategies and cross-validation techniques discussed during the class. The output must be detailed in this notebook.

# Exercise 07.2 (20 points)

The reminder points will be allocated based on the performance of each one submission.

In [1]:
import pandas as pd
import numpy as np

In [2]:
train_df = pd.read_csv('titanic_train.csv', header=0)
test_df = pd.read_csv('titanic_test.csv', header=0)

In [3]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,S


In [4]:
titanic = train_df
titanic.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
titanic.loc[titanic.Embarked.isnull(), 'Embarked'] = titanic.Embarked.mode().values
titanic['Sex_Female'] = (titanic['Sex'] == 'female').astype(np.int)
embarked_dummies = pd.get_dummies(titanic.Embarked, prefix='Embarked')
embarked_dummies.drop(embarked_dummies.columns[0], axis=1, inplace=True)
titanic = pd.concat([titanic, embarked_dummies], axis=1)

In [6]:
# Filtramos los registros que no tienen dato en 'Age'
titanic.ix[(titanic.Age<0),"Na"]=0 
# Los que si tienen dato:
titanic.ix[(titanic.Age>=0),"Na"]=1 
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_Female,Embarked_Q,Embarked_S,Na
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S,0,0,1,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C,1,0,0,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S,1,0,1,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S,1,0,1,1
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,S,0,0,1,1


In [7]:
titanic.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
Sex_Female       0
Embarked_Q       0
Embarked_S       0
Na             177
dtype: int64

In [8]:
data_filter = titanic[titanic['Na'] == 1]
data_filter.head(200)
len(data_filter)

714

In [9]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
feature_cols = ['Survived','Pclass', 'Parch', 'Sex_Female', 'Embarked_Q', 'Embarked_S']
X = data_filter[feature_cols]
y =data_filter.Age
linreg.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [10]:
data_na = titanic[titanic['Na'] != 1]
from sklearn.linear_model import LinearRegression
data_na.head()
X_Na = data_na[feature_cols]
data_na['Age'] = linreg.predict(X_Na)
data_na.head()

/Users/FelipeGarcia/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_Female,Embarked_Q,Embarked_S,Na
5,6,0,3,"Moran, Mr. James",male,29.318544,0,0,330877,8.4583,NaN,Q,0,1,0,NaN
17,18,1,2,"Williams, Mr. Charles Eugene",male,29.308223,0,0,244373,13.0000,NaN,S,0,0,1,NaN
19,20,1,3,"Masselmani, Mrs. Fatima",female,19.350672,0,0,2649,7.2250,NaN,C,1,0,0,NaN
26,27,0,3,"Emir, Mr. Farred Chehab",male,25.873397,0,0,2631,7.2250,NaN,C,0,0,0,NaN
28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,22.795819,0,0,330959,7.8792,NaN,Q,1,1,0,NaN


In [11]:
data_com = pd.DataFrame(data_filter)
    
data_imp = pd.DataFrame (data_na)

titanic = data_com.append(data_imp)
titanic.shape

(891, 16)

In [12]:
titanic.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
Sex_Female       0
Embarked_Q       0
Embarked_S       0
Na             177
dtype: int64

In [13]:
titanic.ix[(titanic.Sex_Female==1) & (titanic.Age<20) ,"Age2"]= "Dama_G1"
titanic.ix[(titanic.Sex_Female==1) & (titanic.Age>20) ,"Age2"]= "Dama_G2"
titanic.ix[(titanic.Sex_Female==0) & (titanic.Age>20) ,"Age2"]= "Hombre_G1"
titanic.ix[(titanic.Sex_Female==0) & (titanic.Age<20) ,"Age2"]= "Hombre_G2"

In [14]:
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_Female,Embarked_Q,Embarked_S,Na,Age2
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S,0,0,1,1,Hombre_G1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C,1,0,0,1,Dama_G2
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S,1,0,1,1,Dama_G2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S,1,0,1,1,Dama_G2
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,S,0,0,1,1,Hombre_G1


In [15]:
Age2_dummies = pd.get_dummies(titanic.Age2, prefix='Age2')
Age2_dummies.drop(Age2_dummies.columns[0], axis=1, inplace=True)
titanic = pd.concat([titanic, Age2_dummies], axis=1)

In [16]:
Age3 = "Age3"
titanic[Age3]=titanic.Age**2

## Train Model

In [17]:
# define X and y
feature_cols = ['Pclass', 'Parch','Age','Sex_Female', 'Embarked_Q', 'Embarked_S','Age2_Dama_G2','Age2_Hombre_G1','Age2_Hombre_G2','Age3']
X = titanic[feature_cols]
y = titanic.Survived

# train/test split
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# train a logistic regression model
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=1e9)
logreg.fit(X_train, y_train)

# make predictions for testing set
y_pred_class = logreg.predict(X_test)

# calculate testing accuracy
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred_class))

0.789237668161


# Apply model to Test set

In [18]:
test =test_df
test.isnull().sum()
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [19]:
test.loc[test.Embarked.isnull(), 'Embarked'] = test.Embarked.mode().values
test.Fare.fillna(titanic.Fare.median(), inplace=True)
test['Sex_Female'] = (test['Sex'] == 'female').astype(np.int)
embarked_dummies = pd.get_dummies(test.Embarked, prefix='Embarked')
embarked_dummies.drop(embarked_dummies.columns[0], axis=1, inplace=True)
test = pd.concat([test, embarked_dummies], axis=1)
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_Female,Embarked_Q,Embarked_S
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0,1,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,1,0,1
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0,1,0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0,0,1
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,1,0,1


In [20]:
test.ix[(test.Age<0),"condicion"]=0 # vacios
test.ix[(test.Age>=0),"condicion"]=1 #Age
test_filter = test[test['condicion'] == 1]
len(test_filter)

332

In [21]:
from sklearn.linear_model import LinearRegression
linregre = LinearRegression()
test_cols = ['Pclass', 'Parch', 'Sex_Female', 'Embarked_Q', 'Embarked_S']
X1 = test_filter[test_cols]
y1 = test_filter.Age
linregre.fit(X1, y1)
linregre.coef_

array([ -8.65900745e+00,  -7.62710669e-01,  -1.81804800e+00,
         4.83848986e+00,   8.01223641e-03])

In [22]:
test_vacios = test[test['condicion'] != 1]
test_vacios.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_Female,Embarked_Q,Embarked_S,condicion
10,902,3,"Ilieff, Mr. Ylio",male,NaN,0,0,349220,7.8958,NaN,S,0,0,1,NaN
22,914,1,"Flegenheim, Mrs. Alfred (Antoinette)",female,NaN,0,0,PC 17598,31.6833,NaN,S,1,0,1,NaN
29,921,3,"Samaan, Mr. Elias",male,NaN,2,0,2662,21.6792,NaN,C,0,0,0,NaN
33,925,3,"Johnston, Mrs. Andrew G (Elizabeth Lily"" Watson)""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,1,0,1,NaN
36,928,3,"Roth, Miss. Sarah A",female,NaN,0,0,342712,8.0500,NaN,S,1,0,1,NaN


In [23]:
from sklearn.linear_model import LinearRegression
X_test = test_vacios[test_cols]
test_vacios['Age'] = linregre.predict(X_test)
test_vacios.head()

/Users/FelipeGarcia/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_Female,Embarked_Q,Embarked_S,condicion
10,902,3,"Ilieff, Mr. Ylio",male,23.546079,0,0,349220,7.8958,NaN,S,0,0,1,NaN
22,914,1,"Flegenheim, Mrs. Alfred (Antoinette)",female,39.046046,0,0,PC 17598,31.6833,NaN,S,1,0,1,NaN
29,921,3,"Samaan, Mr. Elias",male,23.538067,2,0,2662,21.6792,NaN,C,0,0,0,NaN
33,925,3,"Johnston, Mrs. Andrew G (Elizabeth Lily"" Watson)""",female,20.202610,1,2,W./C. 6607,23.4500,NaN,S,1,0,1,NaN
36,928,3,"Roth, Miss. Sarah A",female,21.728031,0,0,342712,8.0500,NaN,S,1,0,1,NaN


In [24]:
completo_test = pd.DataFrame(test_filter)
   
imputados_test = pd.DataFrame (test_vacios)

final_test = completo_test.append(imputados_test)
final_test.shape

(418, 15)

In [25]:
test_df.Age.fillna(titanic.Age.median(), inplace=True)  # Note use median from training set
test_df['Sex_Female'] = (test_df['Sex'] == 'female').astype(np.int)
embarked_dummies = pd.get_dummies(test_df.Embarked, prefix='Embarked')
embarked_dummies.drop(embarked_dummies.columns[0], axis=1, inplace=True)
test_df = pd.concat([test_df, embarked_dummies], axis=1)

In [26]:
test_df.ix[(test_df.Sex_Female==1) & (test_df.Age<20) ,"Age2"]= "Dama_G1"
test_df.ix[(test_df.Sex_Female==1) & (test_df.Age>20) ,"Age2"]= "Dama_G2"
test_df.ix[(test_df.Sex_Female==0) & (test_df.Age>20) ,"Age2"]= "Hombre_G1"
test_df.ix[(test_df.Sex_Female==0) & (test_df.Age<20) ,"Age2"]= "Hombre_G2"
Age2_dummies = pd.get_dummies(test_df.Age2, prefix='Age2')
Age2_dummies.drop(Age2_dummies.columns[0], axis=1, inplace=True)
test_df = pd.concat([test_df, Age2_dummies], axis=1)
Age3 = "Age3"
test_df[Age3]=test_df.Age**2

In [27]:
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_Female,Embarked_Q,Embarked_S,Age2,Age2_Dama_G2,Age2_Hombre_G1,Age2_Hombre_G2,Age3
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0,1,0,Hombre_G1,0,1,0,1190.25
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,1,0,1,Dama_G2,1,0,0,2209.00
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0,1,0,Hombre_G1,0,1,0,3844.00
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0,0,1,Hombre_G1,0,1,0,729.00
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,1,0,1,Dama_G2,1,0,0,484.00


In [28]:
predictions = logreg.predict(test_df[feature_cols])

In [29]:
# Kaggle needs the submission to have a certain format;
# see https://www.kaggle.com/c/titanic-gettingStarted/download/gendermodel.csv
# for an example of what it's supposed to look like.
submission = pd.DataFrame({ 'PassengerId': test_df['PassengerId'],
                            'Survived': predictions })
submission.to_csv("lr.csv", index=False)

# KNN

In [30]:
# Create k-folds
from sklearn.cross_validation import KFold
kf = KFold(X.shape[0], n_folds=20, random_state=0)

results = []
models = []
for train_index, test_index in kf:
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # train a logistic regression model
    models.append(LogisticRegression(C=1e9))
    
    models[-1].fit(X_train, y_train)

    # make predictions for testing set
    y_pred_class = models[-1].predict(X_test)

    # calculate testing accuracy
    results.append(metrics.accuracy_score(y_test, y_pred_class))

In [31]:
results

[0.73333333333333328,
 0.84444444444444444,
 0.82222222222222219,
 0.80000000000000004,
 0.77777777777777779,
 0.84444444444444444,
 0.84444444444444444,
 0.80000000000000004,
 0.75555555555555554,
 0.82222222222222219,
 0.75555555555555554,
 0.72727272727272729,
 0.84090909090909094,
 0.81818181818181823,
 0.81818181818181823,
 0.86363636363636365,
 0.81818181818181823,
 0.81818181818181823,
 0.84090909090909094,
 0.79545454545454541]

In [32]:
probas = pd.DataFrame(index=test_df[feature_cols].index, columns=['p'+ str(i) for i in range(10)])

In [34]:
np.argmax(results)

15

In [35]:
train_index, test_index = list(kf)[15]
X_train, X_test = X.iloc[train_index], X.iloc[test_index]
y_train, y_test = y.iloc[train_index], y.iloc[test_index]

logres = LogisticRegression(C=1e9)
logres.fit(X_train, y_train)
predictions = logreg.predict(test_df[feature_cols])

In [36]:
# Kaggle needs the submission to have a certain format;
# see https://www.kaggle.com/c/titanic-gettingStarted/download/gendermodel.csv
# for an example of what it's supposed to look like.
submission = pd.DataFrame({ 'PassengerId': test_df['PassengerId'],
                            'Survived': predictions })
submission.to_csv("titanic_submission_k15.csv", index=False)

In [52]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=)

In [53]:
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
y_pred_prob = knn.predict_proba(X_test)

In [54]:
(y_pred== y_test).mean()

0.70454545454545459